In [2]:
import os
import json
import shutil
from pathlib import Path
from ultralytics import YOLO
import yaml
from tqdm import tqdm
import random


V3

最优秀的一版结果：58.50

In [13]:
import os
import json
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm
import cv2
from PIL import Image

# --- 1. 优化后的推理配置 ---
CONFIG = {
    # 模型路径
    "model_path": "D:\\43420\\Desktop\\同济大学研一上\\交通数据分析与应用\\第二次作业\\路面病害任务数据集\\road_hazard_yolov8_finetune\\road_hazard_yolov8_finetune\\finetune_v3_unfrozen/weights/best.pt",  # 训练好的模型路径
    "val_img_dir": "val",                    # 验证集图片目录
    "output_json": "submission3.json",        # 输出JSON文件
    "device": 0,                             # GPU编号，CPU用'cpu'     
    # ✅ 优化的推理配置
    "img_size": 1280,                        # 推理图像尺寸
    "conf_threshold": 0.00004,                  # ✅ 从0.001改为0.25（减少假阳性）
    "iou_threshold": 0.655,                   # ✅ 从0.5改为0.65（更严格的NMS）
    
    # ✅ 增强的TTA配置
    "use_tta": False,                         # 启用测试时增强
    "tta_scales": [1.0],    # ✅ 增加4个尺度（原为3个）
    "tta_flips": [0, 1],                     # 翻转类型
}

# YOLO类别索引 -> COCO category_id 的映射
YOLO_TO_COCO = {
    0: 1,  # Pothole -> 1
    1: 2,  # Net -> 2
}

# --- 2. Weighted Boxes Fusion (WBF) 实现 ---
def weighted_boxes_fusion(boxes_list, scores_list, labels_list, 
                          iou_thr=0.55, skip_box_thr=0.0001):
    """
    加权框融合（用于TTA多个预测结果融合）
    比NMS更智能的融合策略
    
    参数:
        boxes_list: list of arrays, 每个array shape=(N, 4), 格式[x1,y1,x2,y2], 归一化坐标
        scores_list: list of arrays, 每个array shape=(N,)
        labels_list: list of arrays, 每个array shape=(N,)
        iou_thr: IoU阈值，判断是否为同一个目标
        skip_box_thr: 跳过置信度太低的框
    
    返回:
        boxes, scores, labels (融合后的结果)
    """
    if len(boxes_list) == 0:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)
    
    # 合并所有预测
    all_boxes = []
    all_scores = []
    all_labels = []
    
    for boxes, scores, labels in zip(boxes_list, scores_list, labels_list):
        if len(boxes) > 0:
            all_boxes.append(boxes)
            all_scores.append(scores)
            all_labels.append(labels)
    
    if len(all_boxes) == 0:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)
    
    all_boxes = np.vstack(all_boxes)
    all_scores = np.hstack(all_scores)
    all_labels = np.hstack(all_labels)
    
    # 按类别分别处理
    unique_labels = np.unique(all_labels)
    fused_boxes = []
    fused_scores = []
    fused_labels = []
    
    for label in unique_labels:
        mask = all_labels == label
        class_boxes = all_boxes[mask]
        class_scores = all_scores[mask]
        
        # 按分数排序
        order = class_scores.argsort()[::-1]
        class_boxes = class_boxes[order]
        class_scores = class_scores[order]
        
        # WBF核心算法
        while len(class_boxes) > 0:
            # 取最高分的框
            best_box = class_boxes[0:1]
            best_score = class_scores[0:1]
            
            # 计算与其他框的IoU
            if len(class_boxes) > 1:
                ious = compute_iou_vectorized(best_box, class_boxes[1:])
                
                # 找到高IoU的框进行融合
                merge_mask = ious >= iou_thr
                merge_indices = np.where(merge_mask)[0] + 1
                
                if len(merge_indices) > 0:
                    # 加权平均框坐标
                    merge_boxes = class_boxes[np.concatenate([[0], merge_indices])]
                    merge_scores = class_scores[np.concatenate([[0], merge_indices])]
                    
                    weights = merge_scores / merge_scores.sum()
                    fused_box = (merge_boxes * weights[:, None]).sum(axis=0)
                    fused_score = merge_scores.mean()  # 平均置信度
                    
                    fused_boxes.append(fused_box)
                    fused_scores.append(fused_score)
                    fused_labels.append(label)
                    
                    # 移除已融合的框
                    keep_mask = np.ones(len(class_boxes), dtype=bool)
                    keep_mask[0] = False
                    keep_mask[merge_indices] = False
                    class_boxes = class_boxes[keep_mask]
                    class_scores = class_scores[keep_mask]
                else:
                    # 没有重叠的框，单独保留
                    fused_boxes.append(best_box[0])
                    fused_scores.append(best_score[0])
                    fused_labels.append(label)
                    class_boxes = class_boxes[1:]
                    class_scores = class_scores[1:]
            else:
                # 只剩最后一个框
                fused_boxes.append(best_box[0])
                fused_scores.append(best_score[0])
                fused_labels.append(label)
                break
    
    if len(fused_boxes) > 0:
        return (np.array(fused_boxes), 
                np.array(fused_scores), 
                np.array(fused_labels))
    else:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)

def compute_iou_vectorized(box, boxes):
    """向量化计算IoU（Intersection over Union）"""
    x1 = np.maximum(box[0, 0], boxes[:, 0])
    y1 = np.maximum(box[0, 1], boxes[:, 1])
    x2 = np.minimum(box[0, 2], boxes[:, 2])
    y2 = np.minimum(box[0, 3], boxes[:, 3])
    
    # 交集面积
    intersection = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    
    # 并集面积
    box_area = (box[0, 2] - box[0, 0]) * (box[0, 3] - box[0, 1])
    boxes_area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    union = box_area + boxes_area - intersection
    
    return intersection / (union + 1e-6)

# --- 3. TTA推理（使用WBF融合）---
def predict_with_tta(model, image_path, scales, flips, img_size, conf_threshold, iou_threshold):
    """
    测试时增强预测
    使用多尺度和翻转，最后用WBF融合结果
    
    参数:
        model: YOLO模型
        image_path: 图片路径
        scales: 缩放尺度列表 [0.8, 1.0, 1.2, 1.4]
        flips: 翻转类型列表 [0=不翻转, 1=水平翻转]
        img_size: 推理尺寸
        conf_threshold: 置信度阈值
        iou_threshold: NMS IoU阈值
    
    返回:
        boxes, scores, labels (numpy arrays, 绝对坐标)
    """
    # 读取原始图片
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    all_boxes_list = []
    all_scores_list = []
    all_labels_list = []
    
    # 遍历所有尺度和翻转组合
    for scale in scales:
        for flip in flips:
            # 准备图片
            test_img = img.copy()
            
            # 水平翻转
            if flip == 1:
                test_img = cv2.flip(test_img, 1)
            
            # 缩放
            if scale != 1.0:
                new_h, new_w = int(h * scale), int(w * scale)
                test_img = cv2.resize(test_img, (new_w, new_h))
            
            # 推理
            results = model.predict(
                test_img,
                imgsz=img_size,
                conf=conf_threshold,
                iou=iou_threshold,
                verbose=False,
                device=CONFIG['device']
            )[0]
            
            if len(results.boxes) == 0:
                continue
            
            # 提取检测结果
            boxes = results.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
            scores = results.boxes.conf.cpu().numpy()
            labels = results.boxes.cls.cpu().numpy().astype(int)
            
            # 坐标还原（缩放）
            if scale != 1.0:
                boxes /= scale
            
            # 坐标还原（翻转）
            if flip == 1:
                boxes[:, [0, 2]] = w - boxes[:, [2, 0]]
            
            # 转换为归一化坐标（用于WBF）
            norm_boxes = boxes.copy()
            norm_boxes[:, [0, 2]] /= w
            norm_boxes[:, [1, 3]] /= h
            norm_boxes = np.clip(norm_boxes, 0, 1)
            
            all_boxes_list.append(norm_boxes)
            all_scores_list.append(scores)
            all_labels_list.append(labels)
    
    # 使用WBF融合所有预测
    if len(all_boxes_list) > 0:
        fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
            all_boxes_list, 
            all_scores_list, 
            all_labels_list,
            iou_thr=0.55,
            skip_box_thr=0.0001
        )
        
        # 反归一化坐标（恢复为绝对坐标）
        if len(fused_boxes) > 0:
            fused_boxes[:, [0, 2]] *= w
            fused_boxes[:, [1, 3]] *= h
        
        return fused_boxes, fused_scores, fused_labels
    else:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)

# --- 4. 标准推理（无TTA）---
def predict_standard(model, image_path, img_size, conf_threshold, iou_threshold):
    """标准推理（不使用TTA，速度快）"""
    results = model.predict(
        str(image_path),
        imgsz=img_size,
        conf=conf_threshold,
        iou=iou_threshold,
        verbose=False,
        device=CONFIG['device']
    )[0]
    
    if len(results.boxes) == 0:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)
    
    boxes = results.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
    scores = results.boxes.conf.cpu().numpy()
    labels = results.boxes.cls.cpu().numpy().astype(int)
    
    return boxes, scores, labels

# --- 5. 主推理函数 ---
def generate_submission_file():
    """生成提交文件"""
    print(f"\n{'='*70}")
    print(f"🔍 YOLOv8 路面缺陷检测推理系统（优化版）")
    print(f"{'='*70}\n")
    
    # 检查模型文件
    if not Path(CONFIG['model_path']).exists():
        print(f"❌ 错误: 模型文件不存在 {CONFIG['model_path']}")
        print(f"💡 提示: 请先运行 train_yolov8_optimized.py 训练模型")
        return
    
    # 检查验证集目录
    if not Path(CONFIG['val_img_dir']).exists():
        print(f"❌ 错误: 验证集目录不存在 {CONFIG['val_img_dir']}")
        return
    
    # 加载模型
    print(f"⏳ 正在加载模型: {CONFIG['model_path']}")
    model = YOLO(CONFIG['model_path'])
    print(f"✅ 模型加载成功!\n")
    
    # 打印配置信息
    print(f"📊 推理配置（已优化）:")
    print(f"   图像尺寸: {CONFIG['img_size']}px")
    print(f"   置信度阈值: {CONFIG['conf_threshold']} ✅ 优化")
    print(f"   IoU阈值: {CONFIG['iou_threshold']} ✅ 优化")
    print(f"   TTA: {'启用' if CONFIG['use_tta'] else '禁用'}")
    if CONFIG['use_tta']:
        print(f"   TTA尺度: {CONFIG['tta_scales']} ✅ 增加")
        print(f"   TTA翻转: {CONFIG['tta_flips']}")
    print()
    
    # 获取验证集图片列表
    val_dir = Path(CONFIG['val_img_dir'])
    image_files = list(val_dir.glob("*.jpg")) + \
                  list(val_dir.glob("*.png")) + \
                  list(val_dir.glob("*.jpeg"))
    
    print(f"📁 找到 {len(image_files)} 张验证集图片\n")
    
    if len(image_files) == 0:
        print(f"⚠️ 警告: 验证集目录为空")
        return
    
    results = []
    
    # 推理循环
    for img_path in tqdm(image_files, desc="🚀 推理进行中"):
        try:
            # 选择推理方式
            if CONFIG['use_tta']:
                boxes, scores, labels = predict_with_tta(
                    model,
                    img_path,
                    scales=CONFIG['tta_scales'],
                    flips=CONFIG['tta_flips'],
                    img_size=CONFIG['img_size'],
                    conf_threshold=CONFIG['conf_threshold'],
                    iou_threshold=CONFIG['iou_threshold']
                )
            else:
                boxes, scores, labels = predict_standard(
                    model,
                    img_path,
                    img_size=CONFIG['img_size'],
                    conf_threshold=CONFIG['conf_threshold'],
                    iou_threshold=CONFIG['iou_threshold']
                )
            
            # 转换为COCO格式并保存结果
            for i in range(len(boxes)):
                x1, y1, x2, y2 = boxes[i]
                width = x2 - x1
                height = y2 - y1
                
                # 转换YOLO类别索引为COCO category_id
                coco_category_id = YOLO_TO_COCO.get(int(labels[i]), int(labels[i]))
                
                results.append({
                    "bbox": [float(x1), float(y1), float(width), float(height)],
                    "category_id": coco_category_id,
                    "file_name": img_path.name,
                    "score": float(scores[i])
                })
        
        except Exception as e:
            print(f"\n⚠️ 处理图片 {img_path.name} 时出错: {str(e)}")
            continue
    
    # 保存推理结果
    print(f"\n💾 正在保存结果到 {CONFIG['output_json']} ...")
    with open(CONFIG['output_json'], 'w') as f:
        json.dump(results, f, indent=4)
    
    # 打印统计信息
    print(f"\n{'='*70}")
    print(f"✅ 推理完成!")
    print(f"{'='*70}")
    print(f"\n📊 统计信息:")
    print(f"   处理图片数: {len(image_files)}")
    print(f"   总检测框数: {len(results)}")
    
    if len(results) > 0:
        scores = [r['score'] for r in results]
        print(f"   平均置信度: {np.mean(scores):.4f}")
        print(f"   最高置信度: {np.max(scores):.4f}")
        print(f"   最低置信度: {np.min(scores):.4f}")
        
        # 按类别统计
        category_counts = {}
        for r in results:
            cat_id = r['category_id']
            category_counts[cat_id] = category_counts.get(cat_id, 0) + 1
        
        print(f"\n📈 类别分布:")
        category_names = {1: "Pothole", 2: "Net"}
        for cat_id, count in sorted(category_counts.items()):
            cat_name = category_names.get(cat_id, f"Class {cat_id}")
            percentage = (count / len(results)) * 100
            print(f"   {cat_name}: {count} 个 ({percentage:.1f}%)")
    
    print(f"\n🎉 结果已保存至: {CONFIG['output_json']}")
    print(f"💡 下一步: 将 {CONFIG['output_json']} 提交到评测平台")

# --- 6. 运行入口 ---
if __name__ == "__main__":
    generate_submission_file()


🔍 YOLOv8 路面缺陷检测推理系统（优化版）

⏳ 正在加载模型: D:\43420\Desktop\同济大学研一上\交通数据分析与应用\第二次作业\路面病害任务数据集\road_hazard_yolov8_finetune\road_hazard_yolov8_finetune\finetune_v3_unfrozen/weights/best.pt
✅ 模型加载成功!

📊 推理配置（已优化）:
   图像尺寸: 1280px
   置信度阈值: 4e-05 ✅ 优化
   IoU阈值: 0.655 ✅ 优化
   TTA: 禁用

📁 找到 462 张验证集图片



🚀 推理进行中: 100%|██████████| 462/462 [00:25<00:00, 18.45it/s]



💾 正在保存结果到 submission3.json ...

✅ 推理完成!

📊 统计信息:
   处理图片数: 462
   总检测框数: 20220
   平均置信度: 0.0269
   最高置信度: 0.9733
   最低置信度: 0.0000

📈 类别分布:
   Pothole: 4625 个 (22.9%)
   Net: 15595 个 (77.1%)

🎉 结果已保存至: submission3.json
💡 下一步: 将 submission3.json 提交到评测平台


In [ ]:
import os
import json
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm
import cv2
from PIL import Image

# --- 1. 优化后的推理配置 ---
CONFIG = {
    # 模型路径
    "model_path": "D:\\43420\\Desktop\\同济大学研一上\\交通数据分析与应用\\第二次作业\\路面病害任务数据集\\road_hazard_yolov8_finetune\\road_hazard_yolov8_finetune\\finetune_v3_unfrozen/weights/best.pt",  # 训练好的模型路径
    "val_img_dir": "val",                    # 验证集图片目录
    "output_json": "submission3.json",        # 输出JSON文件
    "device": 0,                             # GPU编号，CPU用'cpu'     
    # ✅ 优化的推理配置
    "img_size": 1280,                        # 推理图像尺寸
    "conf_threshold": 0.00004,                  # ✅ 从0.001改为0.25（减少假阳性）
    "iou_threshold": 0.659,                   # ✅ 从0.5改为0.65（更严格的NMS）
    
    # ✅ 增强的TTA配置
    "use_tta": False,                         # 启用测试时增强
    "tta_scales": [1.0],    # ✅ 增加4个尺度（原为3个）
    "tta_flips": [0],                     # 翻转类型
}

# YOLO类别索引 -> COCO category_id 的映射
YOLO_TO_COCO = {
    0: 1,  # Pothole -> 1
    1: 2,  # Net -> 2
}

# --- 2. Weighted Boxes Fusion (WBF) 实现 ---
def weighted_boxes_fusion(boxes_list, scores_list, labels_list, 
                          iou_thr=0.55, skip_box_thr=0.0001):
    """
    加权框融合（用于TTA多个预测结果融合）
    比NMS更智能的融合策略
    
    参数:
        boxes_list: list of arrays, 每个array shape=(N, 4), 格式[x1,y1,x2,y2], 归一化坐标
        scores_list: list of arrays, 每个array shape=(N,)
        labels_list: list of arrays, 每个array shape=(N,)
        iou_thr: IoU阈值，判断是否为同一个目标
        skip_box_thr: 跳过置信度太低的框
    
    返回:
        boxes, scores, labels (融合后的结果)
    """
    if len(boxes_list) == 0:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)
    
    # 合并所有预测
    all_boxes = []
    all_scores = []
    all_labels = []
    
    for boxes, scores, labels in zip(boxes_list, scores_list, labels_list):
        if len(boxes) > 0:
            all_boxes.append(boxes)
            all_scores.append(scores)
            all_labels.append(labels)
    
    if len(all_boxes) == 0:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)
    
    all_boxes = np.vstack(all_boxes)
    all_scores = np.hstack(all_scores)
    all_labels = np.hstack(all_labels)
    
    # 按类别分别处理
    unique_labels = np.unique(all_labels)
    fused_boxes = []
    fused_scores = []
    fused_labels = []
    
    for label in unique_labels:
        mask = all_labels == label
        class_boxes = all_boxes[mask]
        class_scores = all_scores[mask]
        
        # 按分数排序
        order = class_scores.argsort()[::-1]
        class_boxes = class_boxes[order]
        class_scores = class_scores[order]
        
        # WBF核心算法
        while len(class_boxes) > 0:
            # 取最高分的框
            best_box = class_boxes[0:1]
            best_score = class_scores[0:1]
            
            # 计算与其他框的IoU
            if len(class_boxes) > 1:
                ious = compute_iou_vectorized(best_box, class_boxes[1:])
                
                # 找到高IoU的框进行融合
                merge_mask = ious >= iou_thr
                merge_indices = np.where(merge_mask)[0] + 1
                
                if len(merge_indices) > 0:
                    # 加权平均框坐标
                    merge_boxes = class_boxes[np.concatenate([[0], merge_indices])]
                    merge_scores = class_scores[np.concatenate([[0], merge_indices])]
                    
                    weights = merge_scores / merge_scores.sum()
                    fused_box = (merge_boxes * weights[:, None]).sum(axis=0)
                    fused_score = merge_scores.mean()  # 平均置信度
                    
                    fused_boxes.append(fused_box)
                    fused_scores.append(fused_score)
                    fused_labels.append(label)
                    
                    # 移除已融合的框
                    keep_mask = np.ones(len(class_boxes), dtype=bool)
                    keep_mask[0] = False
                    keep_mask[merge_indices] = False
                    class_boxes = class_boxes[keep_mask]
                    class_scores = class_scores[keep_mask]
                else:
                    # 没有重叠的框，单独保留
                    fused_boxes.append(best_box[0])
                    fused_scores.append(best_score[0])
                    fused_labels.append(label)
                    class_boxes = class_boxes[1:]
                    class_scores = class_scores[1:]
            else:
                # 只剩最后一个框
                fused_boxes.append(best_box[0])
                fused_scores.append(best_score[0])
                fused_labels.append(label)
                break
    
    if len(fused_boxes) > 0:
        return (np.array(fused_boxes), 
                np.array(fused_scores), 
                np.array(fused_labels))
    else:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)

def compute_iou_vectorized(box, boxes):
    """向量化计算IoU（Intersection over Union）"""
    x1 = np.maximum(box[0, 0], boxes[:, 0])
    y1 = np.maximum(box[0, 1], boxes[:, 1])
    x2 = np.minimum(box[0, 2], boxes[:, 2])
    y2 = np.minimum(box[0, 3], boxes[:, 3])
    
    # 交集面积
    intersection = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    
    # 并集面积
    box_area = (box[0, 2] - box[0, 0]) * (box[0, 3] - box[0, 1])
    boxes_area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    union = box_area + boxes_area - intersection
    
    return intersection / (union + 1e-6)

# --- 3. TTA推理（使用WBF融合）---
def predict_with_tta(model, image_path, scales, flips, img_size, conf_threshold, iou_threshold):
    """
    测试时增强预测
    使用多尺度和翻转，最后用WBF融合结果
    
    参数:
        model: YOLO模型
        image_path: 图片路径
        scales: 缩放尺度列表 [0.8, 1.0, 1.2, 1.4]
        flips: 翻转类型列表 [0=不翻转, 1=水平翻转]
        img_size: 推理尺寸
        conf_threshold: 置信度阈值
        iou_threshold: NMS IoU阈值
    
    返回:
        boxes, scores, labels (numpy arrays, 绝对坐标)
    """
    # 读取原始图片
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    all_boxes_list = []
    all_scores_list = []
    all_labels_list = []
    
    # 遍历所有尺度和翻转组合
    for scale in scales:
        for flip in flips:
            # 准备图片
            test_img = img.copy()
            
            # 水平翻转
            if flip == 1:
                test_img = cv2.flip(test_img, 1)
            
            # 缩放
            if scale != 1.0:
                new_h, new_w = int(h * scale), int(w * scale)
                test_img = cv2.resize(test_img, (new_w, new_h))
            
            # 推理
            results = model.predict(
                test_img,
                imgsz=img_size,
                conf=conf_threshold,
                iou=iou_threshold,
                verbose=False,
                device=CONFIG['device']
            )[0]
            
            if len(results.boxes) == 0:
                continue
            
            # 提取检测结果
            boxes = results.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
            scores = results.boxes.conf.cpu().numpy()
            labels = results.boxes.cls.cpu().numpy().astype(int)
            
            # 坐标还原（缩放）
            if scale != 1.0:
                boxes /= scale
            
            # 坐标还原（翻转）
            if flip == 1:
                boxes[:, [0, 2]] = w - boxes[:, [2, 0]]
            
            # 转换为归一化坐标（用于WBF）
            norm_boxes = boxes.copy()
            norm_boxes[:, [0, 2]] /= w
            norm_boxes[:, [1, 3]] /= h
            norm_boxes = np.clip(norm_boxes, 0, 1)
            
            all_boxes_list.append(norm_boxes)
            all_scores_list.append(scores)
            all_labels_list.append(labels)
    
    # 使用WBF融合所有预测
    if len(all_boxes_list) > 0:
        fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
            all_boxes_list, 
            all_scores_list, 
            all_labels_list,
            iou_thr=0.55,
            skip_box_thr=0.0001
        )
        
        # 反归一化坐标（恢复为绝对坐标）
        if len(fused_boxes) > 0:
            fused_boxes[:, [0, 2]] *= w
            fused_boxes[:, [1, 3]] *= h
        
        return fused_boxes, fused_scores, fused_labels
    else:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)

# --- 4. 标准推理（无TTA）---
def predict_standard(model, image_path, img_size, conf_threshold, iou_threshold):
    """标准推理（不使用TTA，速度快）"""
    results = model.predict(
        str(image_path),
        imgsz=img_size,
        conf=conf_threshold,
        iou=iou_threshold,
        verbose=False,
        device=CONFIG['device']
    )[0]
    
    if len(results.boxes) == 0:
        return np.zeros((0, 4)), np.zeros(0), np.zeros(0)
    
    boxes = results.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
    scores = results.boxes.conf.cpu().numpy()
    labels = results.boxes.cls.cpu().numpy().astype(int)
    
    return boxes, scores, labels

# --- 5. 主推理函数 ---
def generate_submission_file():
    """生成提交文件"""
    print(f"\n{'='*70}")
    print(f"🔍 YOLOv8 路面缺陷检测推理系统（优化版）")
    print(f"{'='*70}\n")
    
    # 检查模型文件
    if not Path(CONFIG['model_path']).exists():
        print(f"❌ 错误: 模型文件不存在 {CONFIG['model_path']}")
        print(f"💡 提示: 请先运行 train_yolov8_optimized.py 训练模型")
        return
    
    # 检查验证集目录
    if not Path(CONFIG['val_img_dir']).exists():
        print(f"❌ 错误: 验证集目录不存在 {CONFIG['val_img_dir']}")
        return
    
    # 加载模型
    print(f"⏳ 正在加载模型: {CONFIG['model_path']}")
    model = YOLO(CONFIG['model_path'])
    print(f"✅ 模型加载成功!\n")
    
    # 打印配置信息
    print(f"📊 推理配置（已优化）:")
    print(f"   图像尺寸: {CONFIG['img_size']}px")
    print(f"   置信度阈值: {CONFIG['conf_threshold']} ✅ 优化")
    print(f"   IoU阈值: {CONFIG['iou_threshold']} ✅ 优化")
    print(f"   TTA: {'启用' if CONFIG['use_tta'] else '禁用'}")
    if CONFIG['use_tta']:
        print(f"   TTA尺度: {CONFIG['tta_scales']} ✅ 增加")
        print(f"   TTA翻转: {CONFIG['tta_flips']}")
    print()
    
    # 获取验证集图片列表
    val_dir = Path(CONFIG['val_img_dir'])
    image_files = list(val_dir.glob("*.jpg")) + \
                  list(val_dir.glob("*.png")) + \
                  list(val_dir.glob("*.jpeg"))
    
    print(f"📁 找到 {len(image_files)} 张验证集图片\n")
    
    if len(image_files) == 0:
        print(f"⚠️ 警告: 验证集目录为空")
        return
    
    results = []
    
    # 推理循环
    for img_path in tqdm(image_files, desc="🚀 推理进行中"):
        try:
            # 选择推理方式
            if CONFIG['use_tta']:
                boxes, scores, labels = predict_with_tta(
                    model,
                    img_path,
                    scales=CONFIG['tta_scales'],
                    flips=CONFIG['tta_flips'],
                    img_size=CONFIG['img_size'],
                    conf_threshold=CONFIG['conf_threshold'],
                    iou_threshold=CONFIG['iou_threshold']
                )
            else:
                boxes, scores, labels = predict_standard(
                    model,
                    img_path,
                    img_size=CONFIG['img_size'],
                    conf_threshold=CONFIG['conf_threshold'],
                    iou_threshold=CONFIG['iou_threshold']
                )
            
            # 转换为COCO格式并保存结果
            for i in range(len(boxes)):
                x1, y1, x2, y2 = boxes[i]
                width = x2 - x1
                height = y2 - y1
                
                # 转换YOLO类别索引为COCO category_id
                coco_category_id = YOLO_TO_COCO.get(int(labels[i]), int(labels[i]))
                
                results.append({
                    "bbox": [float(x1), float(y1), float(width), float(height)],
                    "category_id": coco_category_id,
                    "file_name": img_path.name,
                    "score": float(scores[i])
                })
        
        except Exception as e:
            print(f"\n⚠️ 处理图片 {img_path.name} 时出错: {str(e)}")
            continue
    
    # 保存推理结果
    print(f"\n💾 正在保存结果到 {CONFIG['output_json']} ...")
    with open(CONFIG['output_json'], 'w') as f:
        json.dump(results, f, indent=4)
    
    # 打印统计信息
    print(f"\n{'='*70}")
    print(f"✅ 推理完成!")
    print(f"{'='*70}")
    print(f"\n📊 统计信息:")
    print(f"   处理图片数: {len(image_files)}")
    print(f"   总检测框数: {len(results)}")
    
    if len(results) > 0:
        scores = [r['score'] for r in results]
        print(f"   平均置信度: {np.mean(scores):.4f}")
        print(f"   最高置信度: {np.max(scores):.4f}")
        print(f"   最低置信度: {np.min(scores):.4f}")
        
        # 按类别统计
        category_counts = {}
        for r in results:
            cat_id = r['category_id']
            category_counts[cat_id] = category_counts.get(cat_id, 0) + 1
        
        print(f"\n📈 类别分布:")
        category_names = {1: "Pothole", 2: "Net"}
        for cat_id, count in sorted(category_counts.items()):
            cat_name = category_names.get(cat_id, f"Class {cat_id}")
            percentage = (count / len(results)) * 100
            print(f"   {cat_name}: {count} 个 ({percentage:.1f}%)")
    
    print(f"\n🎉 结果已保存至: {CONFIG['output_json']}")
    print(f"💡 下一步: 将 {CONFIG['output_json']} 提交到评测平台")

# --- 6. 运行入口 ---
if __name__ == "__main__":
    generate_submission_file()


🔍 YOLOv8 路面缺陷检测推理系统（优化版）

⏳ 正在加载模型: D:\43420\Desktop\同济大学研一上\交通数据分析与应用\第二次作业\路面病害任务数据集\road_hazard_yolov8_finetune\road_hazard_yolov8_finetune\finetune_v3_unfrozen/weights/best.pt
✅ 模型加载成功!

📊 推理配置（已优化）:
   图像尺寸: 1280px
   置信度阈值: 4e-05 ✅ 优化
   IoU阈值: 0.659 ✅ 优化
   TTA: 禁用

📁 找到 462 张验证集图片



🚀 推理进行中: 100%|██████████| 462/462 [00:24<00:00, 19.20it/s]



💾 正在保存结果到 submission3.json ...

✅ 推理完成!

📊 统计信息:
   处理图片数: 462
   总检测框数: 20473
   平均置信度: 0.0266
   最高置信度: 0.9733
   最低置信度: 0.0000

📈 类别分布:
   Pothole: 4646 个 (22.7%)
   Net: 15827 个 (77.3%)

🎉 结果已保存至: submission3.json
💡 下一步: 将 submission3.json 提交到评测平台


最好推理代码——59.56

In [27]:
import os
import json
import numpy as np
import cv2
import torch
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm

# --- 增强配置 ---
CONFIG = {
    "model_paths": [
        "D:\\43420\\Desktop\\同济大学研一上\\交通数据分析与应用\\第二次作业\\路面病害任务数据集\\road_hazard_yolov8_finetune\\road_hazard_yolov8_finetune\\finetune_v3_unfrozen/weights/best.pt", 
        # "path/to/second_best.pt" # 如果有多个模型，填在这里实现 Ensemble
    ],
    "val_img_dir": "val",
    "output_json": "submission_high_ap.json",
    "device": 0,
    
    # 刷分关键：多尺度设置
    # 对于小目标，增加 1536 甚至 1920 的尺度；对于大目标，保留 1280
    "inference_sizes": [1024, 1280, 1536], 
    "conf_threshold": 0.001,  # 刷 AP 建议设在 0.001-0.01 之间，太低会导致无效框过多
    "iou_threshold": 0.6,    # NMS 阈值
    "use_wbf": True,         # 使用 WBF 替代普通 NMS
}

YOLO_TO_COCO = {0: 1, 1: 2}

def get_predictions(model, img, imgsz, conf, iou):
    """单次推理封装"""
    results = model.predict(
        img,
        imgsz=imgsz,
        conf=conf,
        iou=iou,
        device=CONFIG['device'],
        verbose=False,
        augment=True # 开启 YOLO 原生的简单 TTA (Flip/Scale)
    )[0]
    
    if len(results.boxes) == 0:
        return None
        
    return {
        "boxes": results.boxes.xyxyn.cpu().numpy(), # 使用归一化坐标方便 WBF
        "scores": results.boxes.conf.cpu().numpy(),
        "labels": results.boxes.cls.cpu().numpy().astype(int)
    }

def run_advanced_inference():
    # 1. 加载所有模型实现集成
    models = [YOLO(p) for p in CONFIG['model_paths']]
    img_files = list(Path(CONFIG['val_img_dir']).glob("*.jpg"))
    
    all_results = []

    for img_path in tqdm(img_files, desc="🔥 High-AP Inference"):
        img_cv = cv2.imread(str(img_path))
        h, w = img_cv.shape[:2]
        
        combined_boxes = []
        combined_scores = []
        combined_labels = []

        # 2. 多模型 x 多尺度 推理
        for model in models:
            for size in CONFIG['inference_sizes']:
                pred = get_predictions(model, img_cv, size, CONFIG['conf_threshold'], CONFIG['iou_threshold'])
                if pred:
                    combined_boxes.append(pred["boxes"])
                    combined_scores.append(pred["scores"])
                    combined_labels.append(pred["labels"])

        if not combined_boxes:
            continue

        # 3. 执行 WBF (Weighted Boxes Fusion)
        # 注意：这里需要安装 ensemble_boxes 库: pip install ensemble-boxes
        # 如果不方便安装，使用你原有的 WBF 函数即可
        from ensemble_boxes import weighted_boxes_fusion
        
        # 展平 list
        boxes_list = combined_boxes
        scores_list = combined_scores
        labels_list = combined_labels
        
        # WBF 处理
        fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
            boxes_list, scores_list, labels_list, 
            weights=None, iou_thr=0.55, skip_box_thr=CONFIG['conf_threshold']
        )

        # 4. 转换回 COCO 格式
        for i in range(len(fused_boxes)):
            x1, y1, x2, y2 = fused_boxes[i]
            # 还原坐标
            x1, x2 = x1 * w, x2 * w
            y1, y2 = y1 * h, y2 * h
            
            all_results.append({
                "file_name": img_path.name,
                "category_id": YOLO_TO_COCO.get(int(fused_labels[i]), int(fused_labels[i])),
                "bbox": [round(float(x1), 2), round(float(y1), 2), 
                         round(float(x2 - x1), 2), round(float(y2 - y1), 2)],
                "score": float(fused_scores[i])
            })

    # 5. 保存结果
    with open(CONFIG['output_json'], 'w') as f:
        json.dump(all_results, f)
    print(f"✅ 任务完成，结果已保存至 {CONFIG['output_json']}")

if __name__ == "__main__":
    run_advanced_inference()

🔥 High-AP Inference: 100%|██████████| 462/462 [02:19<00:00,  3.32it/s]

✅ 任务完成，结果已保存至 submission_high_ap.json


In [2]:
import os
import json
import numpy as np
import cv2
import torch
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm

# --- 增强配置 ---
CONFIG = {
    "model_paths": [
        "D:\\43420\\Desktop\\同济大学研一上\\交通数据分析与应用\\第二次作业\\路面病害任务数据集\\road_hazard_yolov8_finetune\\road_hazard_yolov8_finetune\\finetune_v3_unfrozen/weights/best.pt", 
        # "path/to/second_best.pt" # 如果有多个模型，填在这里实现 Ensemble
    ],
    "val_img_dir": "test",
    "output_json": "submission_test.json",
    "device": 0,
    
    # 刷分关键：多尺度设置
    # 对于小目标，增加 1536 甚至 1920 的尺度；对于大目标，保留 1280
    "inference_sizes": [1024, 1280, 1536], 
    "conf_threshold": 0.001,  # 刷 AP 建议设在 0.001-0.01 之间，太低会导致无效框过多
    "iou_threshold": 0.6,    # NMS 阈值
    "use_wbf": True,         # 使用 WBF 替代普通 NMS
}

YOLO_TO_COCO = {0: 1, 1: 2}

def get_predictions(model, img, imgsz, conf, iou):
    """单次推理封装"""
    results = model.predict(
        img,
        imgsz=imgsz,
        conf=conf,
        iou=iou,
        device=CONFIG['device'],
        verbose=False,
        augment=True # 开启 YOLO 原生的简单 TTA (Flip/Scale)
    )[0]
    
    if len(results.boxes) == 0:
        return None
        
    return {
        "boxes": results.boxes.xyxyn.cpu().numpy(), # 使用归一化坐标方便 WBF
        "scores": results.boxes.conf.cpu().numpy(),
        "labels": results.boxes.cls.cpu().numpy().astype(int)
    }

def run_advanced_inference():
    # 1. 加载所有模型实现集成
    models = [YOLO(p) for p in CONFIG['model_paths']]
    img_files = list(Path(CONFIG['val_img_dir']).glob("*.jpg"))
    
    all_results = []

    for img_path in tqdm(img_files, desc="🔥 High-AP Inference"):
        img_cv = cv2.imread(str(img_path))
        h, w = img_cv.shape[:2]
        
        combined_boxes = []
        combined_scores = []
        combined_labels = []

        # 2. 多模型 x 多尺度 推理
        for model in models:
            for size in CONFIG['inference_sizes']:
                pred = get_predictions(model, img_cv, size, CONFIG['conf_threshold'], CONFIG['iou_threshold'])
                if pred:
                    combined_boxes.append(pred["boxes"])
                    combined_scores.append(pred["scores"])
                    combined_labels.append(pred["labels"])

        if not combined_boxes:
            continue

        # 3. 执行 WBF (Weighted Boxes Fusion)
        # 注意：这里需要安装 ensemble_boxes 库: pip install ensemble-boxes
        # 如果不方便安装，使用你原有的 WBF 函数即可
        from ensemble_boxes import weighted_boxes_fusion
        
        # 展平 list
        boxes_list = combined_boxes
        scores_list = combined_scores
        labels_list = combined_labels
        
        # WBF 处理
        fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
            boxes_list, scores_list, labels_list, 
            weights=None, iou_thr=0.55, skip_box_thr=CONFIG['conf_threshold']
        )

        # 4. 转换回 COCO 格式
        for i in range(len(fused_boxes)):
            x1, y1, x2, y2 = fused_boxes[i]
            # 还原坐标
            x1, x2 = x1 * w, x2 * w
            y1, y2 = y1 * h, y2 * h
            
            all_results.append({
                "file_name": img_path.name,
                "category_id": YOLO_TO_COCO.get(int(fused_labels[i]), int(fused_labels[i])),
                "bbox": [round(float(x1), 2), round(float(y1), 2), 
                         round(float(x2 - x1), 2), round(float(y2 - y1), 2)],
                "score": float(fused_scores[i])
            })

    # 5. 保存结果
    with open(CONFIG['output_json'], 'w') as f:
        json.dump(all_results, f)
    print(f"✅ 任务完成，结果已保存至 {CONFIG['output_json']}")

if __name__ == "__main__":
    run_advanced_inference()

🔥 High-AP Inference: 100%|██████████| 462/462 [04:10<00:00,  1.85it/s]

✅ 任务完成，结果已保存至 submission_test.json


最好——63

In [3]:
import os
import json
import numpy as np
import cv2
import torch
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm

# --- 增强配置 ---
CONFIG = {
    "model_paths": [
        "D:\\43420\\Desktop\\同济大学研一上\\交通数据分析与应用\\第二次作业\\路面病害任务数据集\\road_hazard_yolov8_finetune\\road_hazard_yolov8_finetune\\finetune_v3_unfrozen/weights/best.pt", 
        # "path/to/second_best.pt" # 如果有多个模型，填在这里实现 Ensemble
    ],
    "val_img_dir": "test",
    "output_json": "submission_test1.json",
    "device": 0,
    
    # 刷分关键：多尺度设置
    # 对于小目标，增加 1536 甚至 1920 的尺度；对于大目标，保留 1280
    "inference_sizes": [1024, 1280, 1536, 1920], 
    "conf_threshold": 0.008,  # 刷 AP 建议设在 0.001-0.01 之间，太低会导致无效框过多
    "iou_threshold": 0.6,    # NMS 阈值
    "use_wbf": True,         # 使用 WBF 替代普通 NMS
}

YOLO_TO_COCO = {0: 1, 1: 2}

def get_predictions(model, img, imgsz, conf, iou):
    """单次推理封装"""
    results = model.predict(
        img,
        imgsz=imgsz,
        conf=conf,
        iou=iou,
        device=CONFIG['device'],
        verbose=False,
        augment=True # 开启 YOLO 原生的简单 TTA (Flip/Scale)
    )[0]
    
    if len(results.boxes) == 0:
        return None
        
    return {
        "boxes": results.boxes.xyxyn.cpu().numpy(), # 使用归一化坐标方便 WBF
        "scores": results.boxes.conf.cpu().numpy(),
        "labels": results.boxes.cls.cpu().numpy().astype(int)
    }

def run_advanced_inference():
    # 1. 加载所有模型实现集成
    models = [YOLO(p) for p in CONFIG['model_paths']]
    img_files = list(Path(CONFIG['val_img_dir']).glob("*.jpg"))
    
    all_results = []

    for img_path in tqdm(img_files, desc="🔥 High-AP Inference"):
        img_cv = cv2.imread(str(img_path))
        h, w = img_cv.shape[:2]
        
        combined_boxes = []
        combined_scores = []
        combined_labels = []

        # 2. 多模型 x 多尺度 推理
        for model in models:
            for size in CONFIG['inference_sizes']:
                pred = get_predictions(model, img_cv, size, CONFIG['conf_threshold'], CONFIG['iou_threshold'])
                if pred:
                    combined_boxes.append(pred["boxes"])
                    combined_scores.append(pred["scores"])
                    combined_labels.append(pred["labels"])

        if not combined_boxes:
            continue

        # 3. 执行 WBF (Weighted Boxes Fusion)
        # 注意：这里需要安装 ensemble_boxes 库: pip install ensemble-boxes
        # 如果不方便安装，使用你原有的 WBF 函数即可
        from ensemble_boxes import weighted_boxes_fusion
        
        # 展平 list
        boxes_list = combined_boxes
        scores_list = combined_scores
        labels_list = combined_labels
        
        # WBF 处理
        fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
            boxes_list, scores_list, labels_list, 
            weights=None, iou_thr=0.55, skip_box_thr=CONFIG['conf_threshold']
        )

        # 4. 转换回 COCO 格式
        for i in range(len(fused_boxes)):
            x1, y1, x2, y2 = fused_boxes[i]
            # 还原坐标
            x1, x2 = x1 * w, x2 * w
            y1, y2 = y1 * h, y2 * h
            
            all_results.append({
                "file_name": img_path.name,
                "category_id": YOLO_TO_COCO.get(int(fused_labels[i]), int(fused_labels[i])),
                "bbox": [round(float(x1), 2), round(float(y1), 2), 
                         round(float(x2 - x1), 2), round(float(y2 - y1), 2)],
                "score": float(fused_scores[i])
            })

    # 5. 保存结果
    with open(CONFIG['output_json'], 'w') as f:
        json.dump(all_results, f)
    print(f"✅ 任务完成，结果已保存至 {CONFIG['output_json']}")

if __name__ == "__main__":
    run_advanced_inference()

🔥 High-AP Inference: 100%|██████████| 462/462 [07:15<00:00,  1.06it/s]

✅ 任务完成，结果已保存至 submission_test1.json


In [1]:
import os
import json
import numpy as np
import cv2
import torch
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm

# --- 增强配置 ---
CONFIG = {
    "model_paths": [
        "D:\\43420\\Desktop\\同济大学研一上\\交通数据分析与应用\\第二次作业\\路面病害任务数据集\\road_hazard_yolov8_finetune\\road_hazard_yolov8_finetune\\finetune_v3_unfrozen/weights/best.pt", 
        "D:\\43420\\Desktop\\同济大学研一上\\交通数据分析与应用\\第二次作业\\路面病害任务数据集\\road_hazard_yolov8_finetune\\road_hazard_yolov8_finetune\\finetune_v4_unfrozen/weights/best.pt" # 如果有多个模型，填在这里实现 Ensemble
    ],
    "val_img_dir": "test",
    "output_json": "submission_test2.json",
    "device": 0,
    
    # 刷分关键：多尺度设置
    # 对于小目标，增加 1536 甚至 1920 的尺度；对于大目标，保留 1280
    "inference_sizes": [1024, 1280, 1536, 1920], 
    "conf_threshold": 0.008,  # 刷 AP 建议设在 0.001-0.01 之间，太低会导致无效框过多
    "iou_threshold": 0.6,    # NMS 阈值
    "use_wbf": True,         # 使用 WBF 替代普通 NMS
}

YOLO_TO_COCO = {0: 1, 1: 2}

def get_predictions(model, img, imgsz, conf, iou):
    """单次推理封装"""
    results = model.predict(
        img,
        imgsz=imgsz,
        conf=conf,
        iou=iou,
        device=CONFIG['device'],
        verbose=False,
        augment=True # 开启 YOLO 原生的简单 TTA (Flip/Scale)
    )[0]
    
    if len(results.boxes) == 0:
        return None
        
    return {
        "boxes": results.boxes.xyxyn.cpu().numpy(), # 使用归一化坐标方便 WBF
        "scores": results.boxes.conf.cpu().numpy(),
        "labels": results.boxes.cls.cpu().numpy().astype(int)
    }

def run_advanced_inference():
    # 1. 加载所有模型实现集成
    models = [YOLO(p) for p in CONFIG['model_paths']]
    img_files = list(Path(CONFIG['val_img_dir']).glob("*.jpg"))
    
    all_results = []

    for img_path in tqdm(img_files, desc="🔥 High-AP Inference"):
        img_cv = cv2.imread(str(img_path))
        h, w = img_cv.shape[:2]
        
        combined_boxes = []
        combined_scores = []
        combined_labels = []

        # 2. 多模型 x 多尺度 推理
        for model in models:
            for size in CONFIG['inference_sizes']:
                pred = get_predictions(model, img_cv, size, CONFIG['conf_threshold'], CONFIG['iou_threshold'])
                if pred:
                    combined_boxes.append(pred["boxes"])
                    combined_scores.append(pred["scores"])
                    combined_labels.append(pred["labels"])

        if not combined_boxes:
            continue

        # 3. 执行 WBF (Weighted Boxes Fusion)
        # 注意：这里需要安装 ensemble_boxes 库: pip install ensemble-boxes
        # 如果不方便安装，使用你原有的 WBF 函数即可
        from ensemble_boxes import weighted_boxes_fusion
        
        # 展平 list
        boxes_list = combined_boxes
        scores_list = combined_scores
        labels_list = combined_labels
        
        # WBF 处理
        fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
            boxes_list, scores_list, labels_list, 
            weights=None, iou_thr=0.55, skip_box_thr=CONFIG['conf_threshold']
        )

        # 4. 转换回 COCO 格式
        for i in range(len(fused_boxes)):
            x1, y1, x2, y2 = fused_boxes[i]
            # 还原坐标
            x1, x2 = x1 * w, x2 * w
            y1, y2 = y1 * h, y2 * h
            
            all_results.append({
                "file_name": img_path.name,
                "category_id": YOLO_TO_COCO.get(int(fused_labels[i]), int(fused_labels[i])),
                "bbox": [round(float(x1), 2), round(float(y1), 2), 
                         round(float(x2 - x1), 2), round(float(y2 - y1), 2)],
                "score": float(fused_scores[i])
            })

    # 5. 保存结果
    with open(CONFIG['output_json'], 'w') as f:
        json.dump(all_results, f)
    print(f"✅ 任务完成，结果已保存至 {CONFIG['output_json']}")

if __name__ == "__main__":
    run_advanced_inference()

🔥 High-AP Inference: 100%|██████████| 462/462 [13:10<00:00,  1.71s/it]

✅ 任务完成，结果已保存至 submission_test2.json


In [1]:
import json
import numpy as np
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt

def analyze_dataset(json_path, img_dir):
    """深度分析数据集特征"""
    print(f"\n{'='*70}")
    print(f"📊 数据集深度分析")
    print(f"{'='*70}\n")
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # 1. 类别分布
    categories = [ann['category_id'] for ann in data['annotations']]
    cat_counts = Counter(categories)
    print(f"📈 类别分布:")
    for cat_id, count in cat_counts.items():
        cat_name = "Pothole" if cat_id == 1 else "Net"
        print(f"   {cat_name}(id={cat_id}): {count} 个 ({count/len(categories)*100:.2f}%)")
    
    # 2. 边界框尺寸分析
    bbox_areas = []
    bbox_aspect_ratios = []
    for ann in data['annotations']:
        w, h = ann['bbox'][2], ann['bbox'][3]
        bbox_areas.append(w * h)
        bbox_aspect_ratios.append(w / (h + 1e-6))
    
    print(f"\n📏 边界框统计:")
    print(f"   平均面积: {np.mean(bbox_areas):.2f} px²")
    print(f"   中位数面积: {np.median(bbox_areas):.2f} px²")
    print(f"   最小面积: {np.min(bbox_areas):.2f} px²")
    print(f"   最大面积: {np.max(bbox_areas):.2f} px²")
    print(f"   平均宽高比: {np.mean(bbox_aspect_ratios):.2f}")
    
    # 3. 小目标占比（COCO定义：面积<32²=1024）
    small_obj_count = sum(1 for area in bbox_areas if area < 1024)
    medium_obj_count = sum(1 for area in bbox_areas if 1024 <= area < 96**2)
    large_obj_count = sum(1 for area in bbox_areas if area >= 96**2)
    
    print(f"\n🎯 目标尺度分布:")
    print(f"   小目标(<32px): {small_obj_count} ({small_obj_count/len(bbox_areas)*100:.2f}%)")
    print(f"   中目标(32-96px): {medium_obj_count} ({medium_obj_count/len(bbox_areas)*100:.2f}%)")
    print(f"   大目标(>96px): {large_obj_count} ({large_obj_count/len(bbox_areas)*100:.2f}%)")
    
    # 4. 每张图片的标注数量
    img_to_anns = {}
    for ann in data['annotations']:
        img_id = ann['image_id']
        img_to_anns[img_id] = img_to_anns.get(img_id, 0) + 1
    
    ann_counts = list(img_to_anns.values())
    print(f"\n🖼️ 每张图片的标注数:")
    print(f"   平均: {np.mean(ann_counts):.2f}")
    print(f"   中位数: {np.median(ann_counts):.0f}")
    print(f"   最大: {max(ann_counts)}")
    print(f"   无标注图片: {len(data['images']) - len(img_to_anns)}")
    
    return {
        'class_imbalance': max(cat_counts.values()) / min(cat_counts.values()),
        'small_obj_ratio': small_obj_count / len(bbox_areas),
        'avg_bbox_area': np.mean(bbox_areas)
    }

# 使用示例
if __name__ == "__main__":
    stats = analyze_dataset("train.json", "train")
    
    print(f"\n{'='*70}")
    print(f"💡 优化建议:")
    print(f"{'='*70}")
    
    if stats['class_imbalance'] > 3:
        print(f"⚠️ 类别严重不平衡 (比例{stats['class_imbalance']:.1f}:1)")
        print(f"   → 使用类别加权损失")
    
    if stats['small_obj_ratio'] > 0.5:
        print(f"⚠️ 小目标占比高 ({stats['small_obj_ratio']*100:.1f}%)")
        print(f"   → 使用多尺度训练")
        print(f"   → 增大输入尺寸到1920")
    
    if stats['avg_bbox_area'] < 2000:
        print(f"⚠️ 目标普遍较小 (平均{stats['avg_bbox_area']:.0f}px²)")
        print(f"   → 关闭Mosaic增强")
        print(f"   → 减小随机裁剪")


📊 数据集深度分析

📈 类别分布:
   Net(id=2): 2720 个 (66.49%)
   Pothole(id=1): 1371 个 (33.51%)

📏 边界框统计:
   平均面积: 76206.92 px²
   中位数面积: 30561.00 px²
   最小面积: 297.00 px²
   最大面积: 1511040.00 px²
   平均宽高比: 2.17

🎯 目标尺度分布:
   小目标(<32px): 327 (7.99%)
   中目标(32-96px): 1017 (24.86%)
   大目标(>96px): 2747 (67.15%)

🖼️ 每张图片的标注数:
   平均: 1.36
   中位数: 1
   最大: 10
   无标注图片: 0

💡 优化建议:
